# Wartime posters

The digital collection [Wartime posters](https://kuleuven.limo.libis.be/discovery/collectionDiscovery?vid=32KUL_KUL:KULeuven&collectionId=81411182030001488&lang=en) includes proclamations issued by the German General Government in Belgium during the World War I. It is published and maintained by [KU Leuven Libraries](https://bib.kuleuven.be/english).

This notebook is the result of the collaboration between KU Leuven and the [University of Alicante](https://www.ua.es/), as part of the [Impact Centre of Competence in digitisation](https://www.digitisation.eu/).

### Introduction
In this example we are going to transform the original metadata into Linked Open Data

### Getting started

The first thing that we need to do is importing all the libraries (Python packages) that we will need to analyse the data. Note that we are using as data a CSV file, consisting of rows and columns. In order to work with CSV files, [pandas](https://pandas.pydata.org/) is a popular Python package that or is used for working with data sets. It has functions for analyzing, cleaning, exploring, and manipulating data.

In [31]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import FOAF, RDF, DCTERMS, VOID, DC, SKOS, OWL
import pandas as pd
import datetime

### Transformation to LOD

Note that the domain URL should be updated with a proper domain

In [32]:
domain = 'https://example.org/'

First, we instantiate all the namespaces that we will use when defining the RDF data

In [33]:
g = Graph()
g.bind("foaf", FOAF)
g.bind("rdf", RDF)
g.bind("dcterms", DCTERMS)
g.bind("dc", DC)
g.bind("void", VOID)
g.bind("skos", SKOS)
g.bind("owl", OWL)

schema = Namespace("https://schema.org/")
g.bind("schema", schema)

viaf = Namespace("https://viaf.org/viaf/")
g.bind("viaf", viaf)

wd = Namespace("http://www.wikidata.org/entity/")
g.bind("wd", wd)

We define the resource for the project Wartime posters dataset

In [34]:
dataset = URIRef(domain + "dataset/wartime-posters")

g.add((dataset, RDF.type, schema.Dataset))
g.add((dataset, schema.url, URIRef("https://kuleuven.limo.libis.be/")))
g.add((dataset, schema.description, Literal("Wartime posters with proclamations issued by the German General Government in Belgium during the World War I.")))
g.add((dataset, schema.name, Literal("Wartime posters")))
g.add((dataset, DC.title, Literal("Wartime posters")))
g.add((dataset, schema.license, URIRef('https://creativecommons.org/publicdomain/zero/1.0/')))

now = datetime.datetime.now()
g.add((dataset, schema.dateCreated, Literal(str(now)[:10])))

<Graph identifier=N517a7f115eba4d248337c5a53d494a07 (<class 'rdflib.graph.Graph'>)>

Let's now loop the CSV file to describe the records

In [35]:
posters_data = pd.read_csv("../../../20250506_CaD_jnbworkshop/Metadata-exports/Wartime_posters/20230301_Wartime_posters.csv", skiprows=[1])
print(posters_data)

               MMS ID         Language  \
0    9992133510701488  dut ; fre ; ger   
1    9992133511201488  dut ; fre ; ger   
2    9992133512001488  dut ; fre ; ger   
3    9992133513201488  dut ; fre ; ger   
4    9992133513301488  dut ; fre ; ger   
..                ...              ...   
166  9992136980201488              dut   
167  9992137078901488              dut   
168  9992137079301488              dut   
169  9992137081401488              dut   
170  9992137081601488              dut   

                                            Main title  \
0    Brussel. 13.06.1915. Mededeelingen van het Dui...   
1    Brussel. 12.06.1915. Mededeelingen [02] van he...   
2    Brussel. 11.06.1915. Mededeelingen [02] van he...   
3    Brussel. 11.06.1915. Mededeelingen [01] van he...   
4    Brussel. 04.06.1915. Mededeelingen van het Dui...   
..                                                 ...   
166  E. H. O. 01.12.1916. Mob. Etappen-Kommandantuu...   
167  Gent. 30.08.1916. Mobime E

Let's now transform the data

In [36]:
for index, row in posters_data.iterrows():
    identifier = row['Resolver URL'].split("/")[3]
  
    classtype = 'https://schema.org/CreativeWork'
        
    record = URIRef(domain + "record/" + identifier.strip())
    g.add((record, RDF.type, URIRef(classtype)))
    g.add((record, schema.sourceOrganization, Literal("KU Leuven")))
    g.add((record, schema.isPartOf, dataset))
    g.add((record, schema.identifier, Literal(identifier)))
    g.add((record, schema.datePublished, Literal(row['Date'])))
    g.add((record, schema.name, Literal(row['Main title']))) # authors could be expanded
    g.add((record, schema.url, URIRef(row["Resolver URL"])))
    g.add((record, schema.author, Literal(row["Author (added entry)"])))

    if row["Place of publication/printing"] != "[Plaats van druk niet vermeld]":
        placeId = ""
        if row["Place of publication/printing"] == "Brussel":
            placeId = "Q111901161"
        elif row["Place of publication/printing"] == "Antwerpen":
            placeId = "Q12892"
        elif row["Place of publication/printing"] == "Gent":
            placeId = "Q1296"
        
        place = URIRef(domain + "place/" + placeId)
        g.add((place, RDF.type, URIRef('https://schema.org/Place')))
        g.add((place, OWL.sameAs, URIRef('http://www.wikidata.org/entity/' + placeId)))
        g.add((place, schema.name, Literal(row["Place of publication/printing"])))
        g.add((record, schema.spatial, place))

Let's store the data generated

In [37]:
g.serialize(destination="dataset_wartime_posters.ttl") 

<Graph identifier=N517a7f115eba4d248337c5a53d494a07 (<class 'rdflib.graph.Graph'>)>

If we open the file dataset_wartime_posters.ttl, we will see something like:

```
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix schema: <https://schema.org/> .
@prefix wd: <http://www.wikidata.org/entity/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://example.org/record/IE4984785> a schema:CreativeWork ;
    schema:author "NaN"^^xsd:double ;
    schema:datePublished "1915" ;
    schema:identifier "IE4984785" ;
    schema:isPartOf <https://example.org/dataset/wartime-posters> ;
    schema:name "Brussel. ??.05.1915. Mededeelingen van het Duitsche Generaal-Gouvernement [ondertekend door] Het Generaal-Gouvernement in België" ;
    schema:sourceOrganization "KU Leuven" ;
    schema:url <http://resolver.libis.be/IE4984785/representation> .

<https://example.org/record/IE4984790> a schema:CreativeWork ;
    schema:author "NaN"^^xsd:double ;
    schema:datePublished "1915" ;
    schema:identifier "IE4984790" ;
    schema:isPartOf <https://example.org/dataset/wartime-posters> ;
    schema:name "Brussel. 01.05.1915. Mededeelingen van het Duitsche Generaal-Gouvernement [ondertekend door] Het Generaal-Gouvernement in België" ;
    schema:sourceOrganization "KU Leuven" ;
    schema:url <http://resolver.libis.be/IE4984790/representation> .

...................

<https://example.org/place/Q1296> a schema:Place ;
    owl:sameAs wd:Q1296 ;
    schema:name "Gent" .

<https://example.org/place/Q111901161> a schema:Place ;
    owl:sameAs wd:Q111901161 ;
    schema:name "Brussel" .

<https://example.org/place/Q12892> a schema:Place ;
    owl:sameAs wd:Q12892 ;
    schema:name "Antwerpen" .

<https://example.org/dataset/wartime-posters> a schema:Dataset ;
    dc:title "Wartime posters" ;
    schema:dateCreated "2025-05-02" ;
    schema:description "Wartime posters with proclamations issued by the German General Government in Belgium during the World War I." ;
    schema:license <https://creativecommons.org/publicdomain/zero/1.0/> ;
    schema:name "Wartime posters" ;
    schema:url <https://kuleuven.limo.libis.be/> .
```